In [ ]:
# Copyright 2019 Institut für Nachrichtentechnik, RWTH Aachen University
%matplotlib notebook

from ipywidgets import interact, interactive, fixed, interact_manual

import ipywidgets as widgets
from IPython.display import clear_output, display, HTML, Audio
import matplotlib.pyplot as plt
from scipy import signal # convolution, 

from ient_nb.ient_plots import *
from ient_nb.ient_signals import *

def convolution(s, h):
    # Convolve s and h numerically
    g = signal.convolve(s, h, mode='same')*deltat; #g = g[0:len(s)];
    return g

<div>
    <img src="ient_nb/figures/rwth_ient_logo@2x.png" style="float: right;height: 5em;">
</div>

# Augendiagramm

Zur Veranschaulichung des Augendiagramms wird zunächst [unipolare Amplitudenabtastung](IUE%20Amplitudenabtastung.ipynb) betrieben, um anschließend das Ausgangssignal des Korrelationsfilters am Empfänger, $y(t)$ zu visualisieren.

## Unipolare Amplitudenabtastung

### Sender

Trägersignal

In [ ]:
fs = 1000 # Abtastrate
La = 100 # Anzahl an Symbolen

# Trägersignal
T=1 # Breite des Trägersignals in Sekunden
s = lambda t: rect(t/T-0.5) 
#s = lambda t: tri((t-0.5)*2/T)*np.sqrt(3)

# Zeitachse
Tmax = La*T # Gesamtlänge des Sendesignals in Sekunden
(t,deltat) = np.linspace(-Tmax, Tmax, 2*Tmax*fs, retstep=True) # Zeitachse

# Plot
fig,ax = plt.subplots(1,1); ax.plot(t, s(t)); ax.set_xlim([-0.25,2.25]); 
ax.set_xlabel(r'$\rightarrow t$'); ax.set_xlabel(r'$\rightarrow s(t)$'); ient_grid(ax); ient_axis(ax);

Binäre Nachricht

In [ ]:
a = np.random.randint(0, 2, size=La) # Nachricht
a[0:10]

Sendesignal

In [ ]:
# Sendesignal
m = np.zeros(len(t))
for l in range(len(a)):
    m = m + a[l]*s(t-l*T)

### Kanal

In [ ]:
def simulate_channel(N0, l=len(m)):
     # Konvertierung Leistungsdichte in Leistung (im Diskreten). Einheit Leistungsdichte: Ws=W/Hz, Einheit Leistung: W
    deltaf = 1/fs
    sigma_n_squared = N0/deltaf # Varianz von n(t)
    
    # n(t) Werte sind Gauß-verteilt
    n = np.random.normal(0, np.sqrt(sigma_n_squared), l)
    return n

n = simulate_channel(0.001)

### Empfänger

In [ ]:
def simulate_receiver(n,h):
    # Faltung mit Filter h(t)
    # eigentlich y(t) = [m(t)+n(t)]*h(t); allerdings ist m(t) determiniert und g(t) vorher berechnet worden
    ne = convolution(n, h);
    y = g + ne
    return (y, ne)

h = s(T-t)
g  = convolution(m, h)
y, ne = simulate_receiver(n,h)


## Plot
fig,axs = plt.subplots(1,2); 
ax = axs[0]; ax.plot(t/T, m, 'rwth');
ax.set_xlabel(r'$\rightarrow t/T$', bbox=ient_wbbox); ax.set_ylabel(r'$m(t)$'); 
ax.set_xlim([0,19]); ient_update_ylim(ax, np.concatenate((m,y)), 0.2); ient_grid(ax); ient_axis(ax); 
ax = axs[1]; ax.plot(t/T, y, 'rwth');
ax.set_xlabel(r'$\rightarrow t/T$', bbox=ient_wbbox); ax.set_ylabel(r'$y(t)$'); 
ax.set_xlim([0,19]); ient_update_ylim(ax, np.concatenate((m,y)), 0.2); ient_grid(ax); ient_axis(ax); 

## Augendiagramm

In [ ]:
def eyediagram(x, L, shift=0):
    k_max = int(np.ceil((len(x) - shift)/L)-1)
    x0 = x[shift:-1]
    x0 = x0[0:k_max*L]
    # Chunk
    E = np.reshape(x0, (-1,L)).T
    tE = np.linspace(-T*fs/(L/2), T*fs/(L/2), L)
    return (E, tE)

# Plot
fix,ax = plt.subplots(1,1, figsize=(8,4)); 
@interact(N0=widgets.FloatLogSlider(base=10,min=-5, max=0, step=0.1, value=-5, description=r'$N_0$', readout_format='.5f'))
def update_plot(N0):
    # Simulate everything
    n = simulate_channel(N0)
    y, ne = simulate_receiver(n,h)
    
    # Create eyediagram data
    E,tE = eyediagram(y,int(T*fs*2))
    
    # Plot
    if not ax.lines:
        ax.plot(tE, E, 'rwth');
        ax.set_xlabel(r'$\rightarrow t/T$', bbox=ient_wbbox); ax.set_ylabel(r'$\uparrow y(t)$', bbox=ient_wbbox);
        ax.set_xlim([-1.09,1.19]); ax.set_ylim([-0.19,1.29]); ax.grid(); ient_axis(ax)
    else:
        for it,l in enumerate(ax.lines): l.set_ydata(E[:,it])

This notebook is provided as [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources) (OER). Feel free to use the notebook for your own purposes. The code is licensed under the [MIT license](https://opensource.org/licenses/MIT). 

Please attribute the work as follows: 
*Christian Rohlfing, Übungsbeispiele zur Vorlesung "Informationsübertragung"*, gehalten von Jens-Rainer Ohm, 2019, Institut für Nachrichtentechnik, RWTH Aachen University.